In [ ]:
import tifffile
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider, Dropdown

from biofilm_analysis.preprocess import normalize_local_contrast
from biofilm_analysis.segmentation import compute_mask
from biofilm_analysis.io_utils import load_tif_stack





In [ ]:
from pathlib import Path
from glob import glob

tif_files = sorted(glob('example_data/*.tif'))
file_dropdown = Dropdown(options=tif_files, description='TIF:')

def load_stack(path):
    stack = tifffile.imread(path)
    if stack.shape[0] < stack.shape[-1]:
        stack = np.transpose(stack, (1, 2, 0))
    return stack

stack = load_stack(tif_files[0])
print(f'Loaded stack: {stack.shape}')

In [ ]:
@interact(
    filename=file_dropdown,
    frame=IntSlider(min=0, max=stack.shape[2]-1, step=1, value=0),
    block=IntSlider(min=21, max=201, step=10, value=101),
    sigma=FloatSlider(min=0.5, max=5, step=0.5, value=2),
    thresh=FloatSlider(min=0.0, max=0.1, step=0.001, value=0.04),
)
def tune(filename, frame, block, sigma, thresh):
    stack = load_stack(filename)
    img = stack[..., frame]
    norm = normalize_local_contrast(img, block)
    from scipy.ndimage import gaussian_filter
    blurred = gaussian_filter(norm, sigma=sigma)
    mask = blurred > thresh

    overlay = np.stack([blurred]*3, axis=-1)
    overlay[mask] = [0, 1, 1]  # cyan

    fig, axs = plt.subplots(1, 3, figsize=(12, 4))
    axs[0].imshow(img, cmap='gray')
    axs[0].set_title('Original')
    axs[1].imshow(blurred, cmap='gray')
    axs[1].set_title('Normalized + Blurred')
    axs[2].imshow(overlay)
    axs[2].set_title('Overlay')
    for ax in axs: ax.axis('off')
    plt.tight_layout()
    plt.show()
